google image search

- https://developers.google.com/custom-search/v1/using_rest
- https://developers.google.com/custom-search/v1/cse/list#request

In [1]:
import urllib

import requests
import tqdm

/Users/kyohei.watarai/.pyenv/versions/3.7.0/lib/python3.7/site-packages/requests/__init__.py:91: RequestsDependencyWarning: urllib3 (1.24.1) or chardet (3.0.4) doesn't match a supported version!
  RequestsDependencyWarning)


In [2]:
with open("../../.credentials/google_key.txt") as f:
    subscription_key = f.read().replace('\n','')

In [3]:
with open("../../.credentials/cx.txt") as f:
    cx_key = f.read().replace('\n', '')

In [4]:
search_url = "https://www.googleapis.com/customsearch/v1"
search_term = "出汁巻玉子"
# search_term = "だし巻き玉子"

In [5]:
def get_bing_images(url, term, key, cx, start):
    params  = {"q": term,  "key": key, "cx": cx, "start": start}
    response = requests.get(url, params=params)
    return response.json()

In [6]:
start_index = 1
items = []
for i in tqdm.tqdm(range(10)):
    search_results = get_bing_images(search_url, search_term, subscription_key, cx_key, start_index)
    items.extend(search_results.get('items'))
    start_index = search_results.get('queries').get('nextPage')[0].get('startIndex')

100%|██████████| 10/10 [00:08<00:00,  1.14it/s]


In [7]:
def download_image(cse_image):
    url = cse_image.get('src')
    url = urllib.parse.urlunparse(urllib.parse.urlparse(url)._replace(query=None))
    response = requests.get(url)
    return response.content, url.split('.')[-1] # image data & extend

In [8]:
def save_image(filename, image):
    with open(filename, 'wb') as f:
        f.write(image)

In [9]:
dir_path = 'dashimaki'
counter = 0
for item in tqdm.tqdm(items):
    cse_images = item.get('pagemap').get('cse_image')
    for cse_image in cse_images:
        image, ext = download_image(cse_image)
        filename = "../../images/raw/{0}/dashimaki_{1:04d}.{2}".format(dir_path, counter, ext)
        save_image(filename, image)
        counter += 1

100%|██████████| 100/100 [00:12<00:00,  2.07it/s]


In [10]:
print("DONE")

DONE
